<a href="https://colab.research.google.com/github/metalearning-inisiatif/Struktur-Superlearning/blob/main/Struktur_Utama_Meta_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy import hstack
from numpy import vstack
from numpy import asarray
from sklearn.datasets import make_blobs
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
 

In [3]:
# tentang make_blobs
X, y = make_blobs(n_samples=10, centers=3, n_features=2,random_state=1)
print(X.shape)

(10, 2)


In [4]:

a = np.array((1,2,3))
b = np.array((4,5,6))
print(a)
np.hstack((a,b))

[1 2 3]


array([1, 2, 3, 4, 5, 6])

In [5]:
a = np.array((1, 2, 3))
b = np.array((4, 5, 6))
print(a.shape)
np.vstack((a,b))


(3,)


array([[1, 2, 3],
       [4, 5, 6]])

In [6]:
a = (1, 2)
b=np.asarray(a)
b

array([1, 2])

In [7]:
a=list()
b=list()
d=list()
e=list()
a.append(10)
print(a)
a.append(20)
a.append(5)
print("List a berisi: ",a)
c=np.asarray(a)
print("a diubah menjadi array c: ", c)
b.append(-1)
#fungsi extend bersifat iterable dari append yang berulang ulang.
b.extend([1])
b.extend(a)
print("List b adalah: ",b)
c=hstack(b)
print("dengan hstack b diubah dari list menjadi array :",c)
e=d.append(c)
print("Hasil list d mengappend array c: ", e)

[10]
List a berisi:  [10, 20, 5]
a diubah menjadi array c:  [10 20  5]
List b adalah:  [-1, 1, 10, 20, 5]
dengan hstack b diubah dari list menjadi array : [-1  1 10 20  5]
Hasil list d mengappend array c:  None


In [8]:
print(X)

[[ -7.23731039  -9.03108652]
 [ -8.16550136  -7.00850439]
 [ -7.02266844  -7.57041289]
 [ -8.86394306  -5.05323981]
 [  0.08525186   3.64528297]
 [ -0.79415228   2.10495117]
 [ -1.34052081   4.15711949]
 [-10.32012971  -4.3374029 ]
 [ -2.18773166   3.33352125]
 [ -8.53560457  -6.01348926]]


In [9]:
print(y)

[2 2 2 1 0 0 0 1 0 1]


In [10]:
def get_models():
	models = list()
	models.append(LogisticRegression(solver='liblinear'))
	models.append(DecisionTreeClassifier())
	models.append(SVC(gamma='scale', probability=True))
	models.append(GaussianNB())
	models.append(KNeighborsClassifier())
	models.append(AdaBoostClassifier())
	models.append(BaggingClassifier(n_estimators=10))
	models.append(RandomForestClassifier(n_estimators=10))
	models.append(ExtraTreesClassifier(n_estimators=10))
	return models
 

def get_out_of_fold_predictions1(X, y, models):
	meta_X, meta_y = list(), list()
	i=0
	
	#definisikan dulu model kfold yang mau digunakan
	
	kfold = KFold(n_splits=5, shuffle=True)
 
	#_ix di bawah ini menyatakan index
	# dengan settingan kfold ini maka jumlah data akan dibagi menjadi 10 fold, dimana 1 fold berjumlah 120 data
	# Dari 10 fold ini 9 akan dijadikan training dan 1 sisanya akan dijadikan testing. 
	# Jadi train_ix akan berjumlah 9 x 120 record = 1080 record
	# test_ix akan berjumlah 120 record

	for train_ix, test_ix in kfold.split(X):
		i=i+1
		print("nilai i : ",i)
		#print(train_ix,test_ix)
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		print("nilai test_y : ",test_y)
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		j=0
		for model in models:
			j=j+1
			print("nilai j : ",j)
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			print("yhat  : ",yhat.shape)
			print(yhat)
	 		# Catatan jumlah kolom yhat mengikuti banyaknya kelas dalam dataset. Dalam hal ini 3 
			fold_yhats.append(yhat)
	 
		print("untuk i = ",i," Jumlah baris fold_yhats adalah :", len(fold_yhats)," Jumlah kolom fold_yhats adalah :", len(fold_yhats[0]))
		#print("untuk i = ",i," fold_yhats adalah :", fold_yhats)	
		meta_X.append(hstack(fold_yhats))
	print(meta_X)
	return vstack(meta_X), asarray(meta_y)
 

def fit_base_models(X, y, models):
	for model in models:
		model.fit(X, y)
 
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model
 

def evaluate_models(X, y, models):
	for model in models:
		yhat = model.predict(X)
		acc = accuracy_score(y, yhat)
		print('%s: %.3f' % (model.__class__.__name__, acc*100))
 
#make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
	meta_X = list()
	for model in models:
		yhat = model.predict_proba(X) 
		meta_X.append(yhat)
	meta_X = hstack(meta_X)
	# predict
	return meta_model.predict(meta_X)

In [11]:
def get_out_of_fold_predictions(X, y, models):
	meta_X, meta_y = list(), list()
	#definisikan dulu model kfold yang mau digunakan
	
	kfold = KFold(n_splits=10, shuffle=True)
 
	#_ix di bawah ini menyatakan index
	# dengan settingan kfold ini maka jumlah data akan dibagi menjadi 10 fold, dimana 1 fold berjumlah 120 data
	# Dari 10 fold ini 9 akan dijadikan training dan 1 sisanya akan dijadikan testing. 
	# Jadi train_ix akan berjumlah 9 x 120 record = 1080 record
	# test_ix akan berjumlah 120 record

	for train_ix, test_ix in kfold.split(X):
		
		fold_yhats = list()
		# get data
		train_X, test_X = X[train_ix], X[test_ix]
		train_y, test_y = y[train_ix], y[test_ix]
		
		meta_y.extend(test_y)
		# fit and make predictions with each sub-model
		#Prediction dari setiap model disimpan sebagai attribut/fitur input dari superlearner model  
		for model in models:
			model.fit(train_X, train_y)
			yhat = model.predict_proba(test_X)
			#print("yhat  : ",yhat.shape)
	 		# Catatan jumlah kolom yhat mengikuti banyaknya kelas dalam dataset. 
			fold_yhats.append(yhat)
		#print("untuk i = ",i," Jumlah baris fold_yhats adalah :", len(fold_yhats)," Jumlah kolom fold_yhats adalah :", len(fold_yhats[0]))
		#print("untuk i = ",i," fold_yhats adalah :", fold_yhats)	
		meta_X.append(hstack(fold_yhats))
	#print(meta_X)
	return vstack(meta_X), asarray(meta_y)
 

In [12]:
Model=get_models()
print(Model)

[LogisticRegression(solver='liblinear'), DecisionTreeClassifier(), SVC(probability=True), GaussianNB(), KNeighborsClassifier(), AdaBoostClassifier(), BaggingClassifier(), RandomForestClassifier(n_estimators=10), ExtraTreesClassifier(n_estimators=10)]


In [13]:
Met_X, Met_y = get_out_of_fold_predictions1(X, y, Model)
print(Met_X.shape)
print(Met_y.shape)

nilai i :  1
nilai test_y :  [2 0]
nilai j :  1
yhat  :  (2, 3)
[[3.15018867e-04 3.70016299e-01 6.29668682e-01]
 [7.17877846e-01 2.75384381e-01 6.73777305e-03]]
nilai j :  2
yhat  :  (2, 3)
[[0. 0. 1.]
 [1. 0. 0.]]
nilai j :  3
yhat  :  (2, 3)
[[0.13918388 0.64828841 0.21252771]
 [0.67470714 0.15513988 0.17015298]]
nilai j :  4
yhat  :  (2, 3)
[[3.00211663e-63 3.06293725e-01 6.93706275e-01]
 [1.00000000e+00 2.67907866e-50 0.00000000e+00]]
nilai j :  5
yhat  :  (2, 3)
[[0.  0.6 0.4]
 [0.6 0.4 0. ]]
nilai j :  6
yhat  :  (2, 3)
[[1.61458905e-10 1.97233467e-10 1.00000000e+00]
 [1.00000000e+00 7.67135312e-11 2.35939671e-10]]
nilai j :  7
yhat  :  (2, 3)
[[0. 0. 1.]
 [1. 0. 0.]]
nilai j :  8
yhat  :  (2, 3)
[[0.  0.1 0.9]
 [1.  0.  0. ]]
nilai j :  9
yhat  :  (2, 3)
[[0. 0. 1.]
 [1. 0. 0.]]
untuk i =  1  Jumlah baris fold_yhats adalah : 9  Jumlah kolom fold_yhats adalah : 2
nilai i :  2
nilai test_y :  [2 0]
nilai j :  1
yhat  :  (2, 3)
[[0.00211463 0.55235574 0.44552963]
 [0.70481403 0.289

In [14]:
print(Met_X)

[[3.15018867e-004 3.70016299e-001 6.29668682e-001 0.00000000e+000
  0.00000000e+000 1.00000000e+000 1.39183875e-001 6.48288410e-001
  2.12527715e-001 3.00211663e-063 3.06293725e-001 6.93706275e-001
  0.00000000e+000 6.00000000e-001 4.00000000e-001 1.61458905e-010
  1.97233467e-010 1.00000000e+000 0.00000000e+000 0.00000000e+000
  1.00000000e+000 0.00000000e+000 1.00000000e-001 9.00000000e-001
  0.00000000e+000 0.00000000e+000 1.00000000e+000]
 [7.17877846e-001 2.75384381e-001 6.73777305e-003 1.00000000e+000
  0.00000000e+000 0.00000000e+000 6.74707142e-001 1.55139879e-001
  1.70152979e-001 1.00000000e+000 2.67907866e-050 0.00000000e+000
  6.00000000e-001 4.00000000e-001 0.00000000e+000 1.00000000e+000
  7.67135312e-011 2.35939671e-010 1.00000000e+000 0.00000000e+000
  0.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  1.00000000e+000 0.00000000e+000 0.00000000e+000]
 [2.11462593e-003 5.52355744e-001 4.45529630e-001 0.00000000e+000
  1.00000000e+000 0.00000000e+000 1.0860

In [15]:
# Membangkitkan dataset klasifikasi dengan dua kelas, 2000 record dan 200 features
X, y = make_blobs(n_samples=2000, centers=2, n_features=200, cluster_std=20,random_state=1)

In [16]:
# split
X, X_val, y, y_val = train_test_split(X, y, test_size=0.40)
# dengan pembagian ini, 60%  data (1200) dipakai sebagai data training dan sisanya 800 data dipakai sebagai data untuk validasi.
print('Train', X.shape, y.shape, 'Test', X_val.shape, y_val.shape)

Train (1200, 200) (1200,) Test (800, 200) (800,)


In [17]:
# Inisiasi models dengan get models
models = get_models()

In [18]:
# Jalankan fungsi get out of fold predictions untuk membangkitkan input bagi superlearner
# Prediksi dari base learner akan menjadi atribut bagi superlearner
# jumlah record untuk superlearner adalah jumlah data test setiap Fold dikalikan dengan jumlah Fold. 
# Jadi jika k=10, dalam setiap Fold, data test berjumlah 120. Diperoleh dari 1200/10. Karena ada 10 fold, maka datatest berjumlah 10*120=1200 baris. 

meta_X, meta_y = get_out_of_fold_predictions(X, y, models)
print('Meta ', meta_X.shape, meta_y.shape)

#print(meta_X)
#Jumlah kolom/atribut hasil adalah sebesar 18.
# 18 berasal dari 9 model memprediksi peluang 2 kelas, yaitu kelas 0 dan 1
# Jumlah baris adalah 10*120=1200. 

Meta  (1200, 18) (1200,)


In [19]:
# fit base models
# Tahapan ini adalah membandingkan akurasi antara base models dan superlearner model.
fit_base_models(X, y, models)
# fit the meta model
# Metamodel difit menggunakan k-fold prediction yang dihasilkan oleh base learner.
meta_model = fit_meta_model(meta_X, meta_y)

In [20]:
# evaluate base models
# disini base models dievaluasi keakuratan prediksi kelas.
evaluate_models(X_val, y_val, models)

LogisticRegression: 99.875
DecisionTreeClassifier: 79.250
SVC: 100.000
GaussianNB: 99.875
KNeighborsClassifier: 99.250
AdaBoostClassifier: 98.125
BaggingClassifier: 90.125
RandomForestClassifier: 93.250
ExtraTreesClassifier: 90.875


In [21]:
# evaluate meta model
# Sebelum meta model melakukan prediksi terhadap X_Val, base models diberi kesempatan dulu untuk melakukan 
# Predict_proba terhadap X_Val. Hasil predict_proba ini menjadi input bagi superlearner.

yhat = super_learner_predictions(X_val, models, meta_model)
print('Akurasi Super Learner: %.3f' % (accuracy_score(y_val, yhat) * 100))

Akurasi Super Learner: 99.875
